In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import openai
from dotenv import load_dotenv

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs

from method.ours.utils import create_driver, get_xpath
from method.ours.parse import parse_form
from method.ours.prompts import get_form_context
from method.ours.history import HistoryTable
from method.ours.generation import (
    generate_constraints_for_input_groups,
    generate_value_for_input_group,
    generate_values_for_input_groups,
    fill_form_with_value_table,
    submit_form
)
from method.ours.feedback import (
    get_local_feedback,
    get_global_feedback
)

In [3]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Global Variables
HEADLESS = False
TEXT_EMBEDDING_METHOD = 'ADA' # ['ADA', 'WORD2VEC', 'SPACY']
GRAPH_EMBEDDING_METHOD = 'NODE2VEC' # ['NODE2VEC', 'GCN']

URL = 'https://www.aircanada.com/ca/en/aco/home.html'

In [4]:
def get_to_form(driver):
    driver.get(URL)
    
    time.sleep(2)
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, 'bkmgFlights_tripTypeSelector_M'))
    ).click()

In [50]:
driver = create_driver(HEADLESS)
get_to_form(driver)

In [7]:
form = driver.find_elements(By.TAG_NAME, 'form')[1]

In [8]:
history_table = HistoryTable(
    url=URL,
    xpath=get_xpath(driver, form)
)

In [9]:
for element in form.find_elements(By.TAG_NAME, 'input'):
    try:
        element.clear()
    except:
        pass

# Processing

In [10]:
html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
relation_graph, input_groups = parse_form(
    driver,
    form,
    TEXT_EMBEDDING_METHOD=TEXT_EMBEDDING_METHOD
)

Computing transition probabilities:   0%|          | 0/506 [00:00<?, ?it/s]

100%|██████████████████████████████████████| 45/45 [00:15<00:00,  2.93it/s]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openai/embeddings_utils.py:68: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


# Generation

In [33]:
value_table = generate_constraints_for_input_groups(input_groups)

  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the relevant information section. You must choose your constraints in the format of our modified version of the Jest library in JavaScript. The list of functions in this modified format are:\n1. toBeEqual(value) # the input field value is exactly equal to the given value\n2. toHaveLengthCondition(condition, value) # the length of the input field value matches the given condition\n3. toBeTruthy() # the input field value is truthy and not empty (not false, 0, '', null, undefined, or NaN)\n4. toHaveCompareCondit

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_tripTypeSelector_M')).toBeEqual('M')",
        "role": "assistant"
      }
    }
  ],
  "created": 1689377955,
  "id": "chatcmpl-7cMfT1aIRPEGXMNSEN5oSxR4Fy3SB",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 620,
    "total_tokens": 637
  }
}
  Total running cost: $0.745
  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the relevant informat

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_destination_trip_1'))\n.toBeTruthy()\n.not.toBeEqual(field('bkmgFlights_origin_trip_1'))\n.not.toBeEqual(field('bkmgFlights_origin_trip_2'))",
        "role": "assistant"
      }
    }
  ],
  "created": 1689377961,
  "id": "chatcmpl-7cMfZtZVZKdA9Q8s47bLpDFIekW7s",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 43,
    "prompt_tokens": 676,
    "total_tokens": 719
  }
}
  Total running cost: $0.809
  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining you

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_destination_trip_2'))\n.toBeTruthy()\n.not.toBeEqual(field('bkmgFlights_origin_trip_2'))",
        "role": "assistant"
      }
    }
  ],
  "created": 1689377972,
  "id": "chatcmpl-7cMfktUt16jTtV2pH6VBn9eCKBuVQ",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 29,
    "prompt_tokens": 676,
    "total_tokens": 705
  }
}
  Total running cost: $0.877
  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input f

In [34]:
value_table = generate_values_for_input_groups(input_groups, value_table)

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, and not the ones in the relevant information section."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for the functionality of the form:\n     Round-trip ,  One-way ,  Multi-city/Stopover ,  Book with points , From, To, Travel dates, From, To, Travel dates\n    We are generating filling values for the following input field:\n    input: <input id=\"bkmgFlights_tripTypeSel

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Los Angeles",
        "role": "assistant"
      }
    }
  ],
  "created": 1689377984,
  "id": "chatcmpl-7cMfwEktiSJMuf9TQy8Ukr3w3F5Oj",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 2,
    "prompt_tokens": 289,
    "total_tokens": 291
  }
}
  Total running cost: $0.942
  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question

In [35]:
fill_form_with_value_table(driver, value_table, input_groups)

<input>value is R</input> at y: (567, 597), x: (219, 319)
R
<input>value is O</input> at y: (567, 597), x: (334, 421)
O
<input>value is M</input> at y: (567, 597), x: (437, 591)
M
<input></input> at y: (574, 594), x: (652, 905)
true
<input></input> at y: (682, 732), x: (218, 538)
New York
<input></input> at y: (682, 732), x: (608, 928)
Los Angeles
<input>DD/MM</input> at y: (682, 732), x: (958, 1288)
20/12
<input></input> at y: (818, 868), x: (218, 538)
Chicago
<input></input> at y: (818, 868), x: (608, 928)
New York
<input>DD/MM</input> at y: (818, 868), x: (958, 1288)
21/12


In [36]:
submit_form(driver, input_groups)

0
1
2
3
4


In [37]:
new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

In [38]:
history_table.add(
    value_table.get_values_dict(),
    'base',
    global_feedback,
    driver.current_url
)

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


# Feedback

In [39]:
new_form = driver.find_elements(By.TAG_NAME, 'form')[1]

In [40]:
new_relation_graph, new_input_groups = parse_form(
    driver,
    new_form,
    prev_relation_graph=relation_graph,
    TEXT_EMBEDDING_METHOD=TEXT_EMBEDDING_METHOD
)

Computing transition probabilities:   0%|          | 0/574 [00:00<?, ?it/s]

100%|██████████████████████████████████████| 55/55 [00:17<00:00,  3.17it/s]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openai/embeddings_utils.py:68: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


In [41]:
global_feedback = get_global_feedback(html, new_html, remove_form_children=True)

In [42]:
global_feedback

['It is not possible to search for flights which have both an origin and a destination in the United States.',
 'It is not possible to book a flight segment between two cities in the same country, unless that country is Canada.  Please contact Air Canada ReservationsOpens in a new tab for assistance with this type of booking.']

In [43]:
value_table = generate_constraints_for_input_groups(
    new_input_groups,
    value_table,
    global_feedback
)

  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the relevant information section. You must choose your constraints in the format of our modified version of the Jest library in JavaScript. The list of functions in this modified format are:\n1. toBeEqual(value) # the input field value is exactly equal to the given value\n2. toHaveLengthCondition(condition, value) # the length of the input field value matches the given condition\n3. toBeTruthy() # the input field value is truthy and not empty (not false, 0, '', null, undefined, or NaN)\n4. toHaveCompareCondit

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_tripTypeSelector_M')).toBeEqual('M')",
        "role": "assistant"
      }
    }
  ],
  "created": 1689378098,
  "id": "chatcmpl-7cMhmhP8SqXzribyCGkckoAlZuOOC",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 770,
    "total_tokens": 787
  }
}
  Total running cost: $1.048
  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the relevant informat

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_destination_trip_1'))\n.toBeTruthy()\n.not.toBeEqual(field('bkmgFlights_origin_trip_1'))\n.not.toBeEqual(field('bkmgFlights_origin_trip_2'))\n.freeTextConstraint('The destination cannot be in the same country as the origin, unless that country is Canada')",
        "role": "assistant"
      }
    }
  ],
  "created": 1689378107,
  "id": "chatcmpl-7cMhvP8uYp1MqSSmVDwOCB8H1qLfe",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 65,
    "prompt_tokens": 835,
    "total_tokens": 900
  }
}
  Total running cost: $1.128
  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, 

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_origin_trip_2'))\n.toBeTruthy()\n.toBeAlphabetical()\n.not.toBeEqual(field('bkmgFlights_origin_trip_1'))\n.not.toBeEqual(field('bkmgFlights_destination_trip_1'))\n.not.toBeEqual(field('bkmgFlights_destination_trip_2'))\n.freeTextConstraint('The origin city must not be in the United States, unless the destination is in Canada')",
        "role": "assistant"
      }
    }
  ],
  "created": 1689378114,
  "id": "chatcmpl-7cMi27ZAGla4Yg0B5R4MUEV5yUXyA",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 84,
    "prompt_tokens": 858,
    "total_tokens": 942
  }
}
  Total running cost: $1.186
  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or ad

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_travelDates_2'))\n.toBeTruthy()\n.toHaveLengthCondition('=', 5)\n.toContainChar('/')\n.toHaveCompareCondition('>', field('bkmgFlights_travelDates_1'))",
        "role": "assistant"
      }
    }
  ],
  "created": 1689378126,
  "id": "chatcmpl-7cMiEGXDCsYwfUAQzwQYvVPoAw8L2",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 45,
    "prompt_tokens": 759,
    "total_tokens": 804
  }
}
  Total running cost: $1.240


In [44]:
value_table = generate_values_for_input_groups(new_input_groups, value_table)

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, and not the ones in the relevant information section."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for the functionality of the form:\n     Round-trip ,  One-way ,  Multi-city/Stopover ,  Book with points , From, To, Travel dates, From, To, Travel dates\n    We are generating filling values for the following input field:\n    input: <input id=\"bkmgFlights_tripTypeSel

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Paris",
        "role": "assistant"
      }
    }
  ],
  "created": 1689378139,
  "id": "chatcmpl-7cMiRWqvYTgPuslicTyLuehAPMYKa",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 322,
    "total_tokens": 323
  }
}
  Total running cost: $1.286
  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, and 

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "21/12",
        "role": "assistant"
      }
    }
  ],
  "created": 1689378142,
  "id": "chatcmpl-7cMiUgXndUbegGUreZlDuF1NkGIGQ",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 3,
    "prompt_tokens": 321,
    "total_tokens": 324
  }
}
  Total running cost: $1.327


In [45]:
fill_form_with_value_table(driver, value_table, new_input_groups)

<input>value is R</input> at y: (743, 773), x: (219, 319)
R
<input>value is O</input> at y: (743, 773), x: (334, 421)
O
<input>value is M</input> at y: (743, 773), x: (437, 591)
M
<input></input> at y: (750, 770), x: (652, 905)
true
<input></input> at y: (858, 908), x: (218, 538)
London
<input></input> at y: (858, 908), x: (608, 928)
Paris
<input>DD/MM</input> at y: (858, 908), x: (958, 1288)
20/12
<input></input> at y: (994, 1044), x: (218, 538)
Tokyo
<input></input> at y: (994, 1044), x: (608, 928)
Berlin
<input>DD/MM</input> at y: (994, 1044), x: (958, 1288)
21/12


In [46]:
submit_form(driver, new_input_groups)

0
1
2


In [47]:
new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

history_table.add(
    value_table.get_values_dict(),
    'base',
    global_feedback,
    driver.current_url
)

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


# Constraint Negation

In [48]:
form_context = get_form_context(input_groups)

In [51]:
get_attr = lambda x, a, d: x[a] if a in x else d

for field_id, entry in value_table.entries.items():
    input_type = get_attr(entry.input_group.node.element.attrs, 'type', 'text')
    if input_type in ['radio', 'checkbox']:
        continue
    
    for i in range(len(entry.constraints)):
        get_to_form(driver)
        
        value_table_copy = value_table.copy()
    
        value_table_copy.entries[field_id].constraints = [value_table.entries[field_id].constraints[i].copy()]
        value_table_copy.entries[field_id].constraints[0].flip_negative()


        input_group = value_table_copy.entries[field_id].input_group

        value_table_copy = generate_value_for_input_group(
            input_group,
            value_table_copy,
            form_context=form_context
        )

        fill_form_with_value_table(driver, value_table_copy, input_groups)

        submit_form(driver, input_groups)
        
        new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
        global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

        history_table.add(
            value_table_copy.get_values_dict(),
            str(type(value_table_copy.entries[field_id].constraints[0])),
            global_feedback,
            driver.current_url
        )
        
        if len(global_feedback) == 0:
            value_table.entries[field_id].constraints[i].approve()

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, and not the ones in the relevant information section."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for the functionality of the form:\n     Round-trip ,  One-way ,  Multi-city/Stopover ,  Book with points , From, To, Travel dates, From, To, Travel dates\n    We are generating filling values for the following input field:\n    input: <input id=\"bkmgFlights_origin_trip

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, and not the ones in the relevant information section."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for the functionality of the form:\n     Round-trip ,  One-way ,  Multi-city/Stopover ,  Book with points , From, To, Travel dates, From, To, Travel dates\n    We are generating filling values for the following input field:\n    input: <input id=\"bkmgFlights_origin_trip

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, and not the ones in the relevant information section."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for the functionality of the form:\n     Round-trip ,  One-way ,  Multi-city/Stopover ,  Book with points , From, To, Travel dates, From, To, Travel dates\n    We are generating filling values for the following input field:\n    input: <input id=\"bkmgFlights_origin_trip

...

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, and not the ones in the relevant information section."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for the functionality of the form:\n     Round-trip ,  One-way ,  Multi-city/Stopover ,  Book with points , From, To, Travel dates, From, To, Travel dates\n    We are generating filling values for the following input field:\n    input: <input id=\"bkmgFlights_travelDates

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, and not the ones in the relevant information section."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for the functionality of the form:\n     Round-trip ,  One-way ,  Multi-city/Stopover ,  Book with points , From, To, Travel dates, From, To, Travel dates\n    We are generating filling values for the following input field:\n    input: <input id=\"bkmgFlights_travelDates

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


In [52]:
history_table.to_test_case('aoc-multitrip.py')

In [57]:
len(set('FEEDBACK-DELIMITER'.join(history_table.table.feedback).split('FEEDBACK-DELIMITER')))

12